In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib einops huggingface_hub trl
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
  Installing build dependencies ..

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer

In [ ]:

dataset_id = "LinhDuong/chatdoctor-200k"
dataset = load_dataset(dataset_id, split="train")
dataset = dataset.remove_columns("instruction")


In [ ]:
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 207408
})

In [ ]:
dataset['input'][3]

'my sone has left sided abd pain..lt pelvic and rt pelvic pain in the groin area.. can only stand for a short time and sitting impossible for the pain..ct showed inlarged lymph nodes..slight elevation in wt count. he has been 10 weeks and unable to get any relief other than rest and pain meds.. he is 32 6ft 3..approx 220 lb'

In [ ]:
# Create an empty list to store the new 'text' column
new_texts = []

# Iterate through each row in the dataset and combine 'input' and 'output' into 'text'
for row in dataset:
    patient_input = row['input']
    assistant_output = row['output']
    new_text = f"Patient : {patient_input}\nAssistant : {assistant_output}"
    new_texts.append(new_text)


In [ ]:
#### Modify this to aggreagate the inout and output in the same column !!!!!
# Create a new dataset with the 'text' column
# new_dataset = dataset.rename_column('input', 'text')

# Create a pandas DataFrame with 'text' column
df = pd.DataFrame({'text': new_texts})


In [ ]:
df

,text
0,Patient : i had what feels like a muscle cramp...
1,Patient : I woke up this morning feeling the w...
2,Patient : My baby has been pooing 5-6 times a ...
3,Patient : my sone has left sided abd pain..lt ...
4,Patient : My one year old son was bit in face ...
...,...
207403,Patient : My 5 year old son has been coughing ...
207404,Patient : My toes on right foot more than left...
207405,Patient : I was diagnosis with pleurisy last T...
207406,Patient : hi i have just found out im pregnant...


In [ ]:
# Sample 20,000 rows randomly from the original DataFrame
sample_size = 20000
randomly_selected_df = df.sample(n=sample_size, random_state=42)

# Reset the index of the new DataFrame
randomly_selected_df = randomly_selected_df.reset_index(drop=True)

# Print the first few rows of the new DataFrame
randomly_selected_df.head()

In [ ]:
# Convert the train_data and test_data DataFrames to datasets.Dataset
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 207408
})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
  model_id, load_in_4bit=True, torch_dtype=torch.float16, device_map="auto"
)

model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_int8_training(model)
peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, peft_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 8388608 || all params: 3508809728 || trainable%: 0.23907275259355415


In [ ]:

training_args = TrainingArguments(
    output_dir="Llama-2-7b-chat-hf-tuned-medical-qa",
    per_device_train_batch_size=4,
    optim="adamw_torch",
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    num_train_epochs=1,
    save_strategy="epoch",
    push_to_hub=True,
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
    packing=True,
    peft_config=peft_config,
)


Cloning https://huggingface.co/Ali-C137/Llama-2-7b-chat-hf-tuned-medical-qa into local empty directory.


In [ ]:

trainer.train()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
100,2.959200
200,2.740400
300,2.567000
400,2.485600
500,2.420200
600,2.385700
700,2.369400
800,2.322500
900,2.304800
1000,2.312700


In [ ]:

trainer.push_to_hub()


NameError: ignored

In [ ]:
model.push_to_hub("Ali-C137/Llama-2-7b-chat-hf-tuned-medical-qa", use_auth_token=True)